In [1]:
import pandas as pd

In [12]:
ADME_public_set = pd.read_csv('ADME_public_set_3521.csv')
smiles = ADME_public_set['SMILES']
ADME_public_set.columns

Index(['Internal ID', 'Vendor ID', 'SMILES', 'CollectionName',
       'LOG HLM_CLint (mL/min/kg)', 'LOG MDR1-MDCK ER (B-A/A-B)',
       'LOG SOLUBILITY PH 6.8 (ug/mL)',
       'LOG PLASMA PROTEIN BINDING (HUMAN) (% unbound)',
       'LOG PLASMA PROTEIN BINDING (RAT) (% unbound)',
       'LOG RLM_CLint (mL/min/kg)'],
      dtype='object')

In [20]:
#get subset where solubility is not NaN
Solubility_set = ADME_public_set.dropna(subset=['LOG SOLUBILITY PH 6.8 (ug/mL)'])
len(Solubility_set)
smiles = Solubility_set['SMILES']
Solubility_set
internal_ids = Solubility_set['Internal ID']
vendors_ids = Solubility_set['Vendor ID']

In [23]:
from rdkit.Chem import rdDistGeom
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdmolops
from tqdm import tqdm
mols_3d = []

for smi,iid,vid in tqdm(zip(smiles, internal_ids, vendors_ids)):
    mol = Chem.AddHs(Chem.MolFromSmiles(smi))
    AllChem.EmbedMolecule(mol, enforceChirality=True, randomSeed=0xf00d)
    rdmolops.AssignStereochemistryFrom3D(mol)
    used_smiles = Chem.MolToSmiles(mol, isomericSmiles=True)
    mol.SetProp("_Name", smi)
    mol.UpdatePropertyCache(strict=False)
    mols_3d.append(mol)
    molblock = Chem.MolToMolBlock(mol)
    print(molblock,file=open(f'/localhome/cschiebroek/MDFPs/mdfptools/carl/biogen_sidequest/mols_3d_biogen/{iid}.mol','w+'))


2173it [00:54, 39.59it/s]
